In [1]:
import math
import numpy as np
import pandas as pd
import plotly.express as px
from python_module.pricing_model import BSMModel, SABRModel, HestonHullWhiteModel


pd.options.display.max_rows = 30
pd.options.display.max_columns = 30
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
def compute_backtest(price_ts, vol_target=0.3, option_type='call', strike_pct=1):
    
    bt_df = price_ts.to_frame('F')
    bt_df['K'] = bt_df['F'].iloc[0] * strike_pct
    bt_df['count'] = 1
    bt_df['days_to_maturity'] = bt_df['count'].iloc[::-1].cumsum().iloc[::-1] - 1
    bt_df['T'] = bt_df['days_to_maturity'] / 252

    rows = []
    for index, row in bt_df.iterrows():
        
        row_dict = row.to_dict()

        row_dict['date'] = index
        F = row_dict['F']
        K = row_dict['K']
        T = row_dict['T']

        pricing_results = BSMModel.compute_option_with_forward(
            F=F,
            K=K,
            T=T,
            r=0,
            sigma=vol_target,
            option_type=option_type,
            compute_greeks=True
            )
        merged_dict = {**row_dict, **pricing_results}
        rows.append(merged_dict)
    bt_df = pd.DataFrame(rows)
    bt_df['dP'] = bt_df['price'].diff()
    bt_df['dH'] = bt_df['F'].diff() * bt_df['delta'].shift(1)

    # Extra outputs
    bt_df['dP_cumsum'] = bt_df['dP'].cumsum()
    bt_df['dH_cumsum'] = bt_df['dH'].cumsum()
    bt_df['premium'] = bt_df['price'].iloc[0]
    bt_df['F0'] = bt_df['F'].iloc[0]
    return bt_df

In [91]:
# Inputs
vol_target = 0.3
rolling_window = 20

In [92]:
# Load data
df_init = pd.read_csv('data/SPY.csv', index_col=0, parse_dates=True)

# Compute the vol target price
df = df_init.copy()
df['log_return'] = np.log(df['price'] / df['price'].shift(1))
df['rolling_std'] = df['log_return'].rolling(window=rolling_window).std() * np.sqrt(252)
df['leverage'] = vol_target / df['rolling_std']
df = df.dropna()
df['vt_price'] = ((df['price'].pct_change() * df['leverage'].shift(1)).fillna(0).add(1).cumprod()) * 100

In [144]:
first_days = [df['vt_price'].index[df['vt_price'].index.year == year][0] for year in df['vt_price'].index.year.unique()]
#temp = df[df['leverage'] < 2]
#first_days = temp.groupby(temp.index.year).apply(lambda g: g.index.min())
first_days = ['2011-01-03']

In [145]:
global_results = dict()
for date in first_days:
    price_ts = df['vt_price'].loc[date:].iloc[:253]
    price_ts = price_ts.pct_change().fillna(0).add(1).cumprod()*100
    atm_call = compute_backtest(price_ts, vol_target=vol_target, option_type='call', strike_pct=1)
    atm_put = compute_backtest(price_ts, vol_target=vol_target, option_type='put', strike_pct=1)
    up_call = compute_backtest(price_ts, vol_target=vol_target, option_type='call', strike_pct=1.2)
    down_put = compute_backtest(price_ts, vol_target=vol_target, option_type='put', strike_pct=0.8)
    strategy_cumsum_pnl = atm_call['dH_cumsum'] + atm_put['dH_cumsum'] - up_call['dH_cumsum'] - down_put['dH_cumsum']
    bt_result = strategy_cumsum_pnl.describe().to_dict()
    bt_result['last'] = strategy_cumsum_pnl.iloc[-1]
    global_results[date] = bt_result

In [146]:
px.line(price_ts)

In [147]:
px.line(strategy_cumsum_pnl)

In [142]:
px.scatter(pd.DataFrame(global_results).loc['max'])

In [143]:
pd.DataFrame(global_results)

,2007-02-01,2008-01-02,2009-01-02,2010-01-04,2011-01-03,2012-01-03,2013-01-02,2014-01-02,2015-01-02,2016-01-04,2017-01-03,2018-01-02,2019-01-02,2020-01-02,2021-01-04,2022-01-03,2023-01-03,2024-01-02,2025-01-02
count,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,252.00,164.00
mean,-2.82,2.00,-0.36,-0.65,-2.80,1.15,1.63,-1.17,-1.59,-2.70,3.04,-1.49,0.86,-1.29,2.32,2.32,1.03,2.71,-1.57
std,2.50,2.33,1.52,2.45,2.87,2.11,1.98,1.61,1.51,2.99,2.14,2.72,1.74,1.76,2.17,1.70,1.89,2.28,2.11
min,-8.36,-1.09,-3.45,-3.88,-13.05,-1.72,-1.00,-8.55,-6.02,-9.59,-0.20,-9.09,-3.51,-6.28,-0.65,-0.33,-3.47,-0.79,-8.02
25%,-4.50,0.10,-1.39,-2.46,-3.89,-0.09,-0.03,-1.47,-2.49,-3.49,1.03,-1.94,-0.09,-2.72,0.11,0.62,-0.48,0.28,-2.83
50%,-2.63,1.22,-0.41,-0.91,-1.80,0.41,0.92,-0.78,-1.28,-1.31,2.83,-1.15,0.35,-1.05,2.26,2.25,0.19,3.02,-0.68
75%,-1.19,4.93,0.45,-0.02,-0.59,3.20,2.67,-0.30,-0.32,-0.56,5.55,-0.01,1.81,-0.05,4.35,4.18,2.45,5.11,-0.13
max,3.30,5.57,3.90,5.43,0.27,5.48,5.26,2.87,1.23,0.71,5.91,3.41,4.70,2.96,5.37,4.94,4.77,5.38,2.54
last,1.55,5.57,3.90,5.43,-13.05,5.48,5.26,-2.97,-0.25,0.71,5.91,-5.74,4.70,2.95,5.37,4.94,4.77,5.38,-2.82


In [105]:
# ...existing code...
fig = px.line(df['leverage'])

# Add vertical lines for each year
for year in df.index.year.unique():
    fig.add_vline(x=str(year), line_width=1, line_dash="dash", line_color="green")

fig.show()